In [3]:
from neo4j import GraphDatabase
import pandas as pd

host = 'bolt://localhost:7687'
user = 'neo4j'
password = 'letmein'
driver = GraphDatabase.driver(host,auth=(user, password))

def neo4j_query(query, params=None):
    with driver.session() as session:
        result = session.run(query, params)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())

In [ ]:
CALL apoc.warmup.run

In [ ]:
CALL apoc.meta.stats
YIELD nodeCount, relCount, labels, relTypesCount

In [ ]:
# NFTs

In [ ]:
MATCH (n:NFT)<-[:FOR_NFT]-(t:Transaction)
WHERE exists (n.Price_USD)
WITH n, t.Price_USD as price
ORDER BY price DESC LIMIT 5
RETURN n.ID_token as token_id, n.Image_url_1 as image_url, price

In [ ]:
MATCH (n:NFT)<-[:FOR_NFT]-(t:Transaction)
WHERE exists(t.Price_USD)
WITH n, t.Price_USD as price
ORDER BY price DESC LIMIT 5
RETURN n.ID_token as token_id, n.Image_url_1 as image_url, price

In [ ]:
MATCH (n:NFT)<-[:FOR_NFT]-(t:Transaction)
WHERE t.Price_USD > 1000000
RETURN count(*)

In [ ]:
MATCH (collection)<-[:IN_COLLECTION]-(n:NFT)<-[:FOR_NFT]-(t:Transaction)
WHERE exists (t.Price_USD)
RETURN collection.Collection_cleaned AS collection, 
       avg(t.Price_USD) AS averagePrice, 
       count(distinct n) AS numberOfNfts
ORDER BY averagePrice DESC LIMIT 5

In [ ]:
# Transactions

In [ ]:
MATCH (n:Transaction)
RETURN n.Crypto AS crypto, count(*) AS count
ORDER BY count DESC
LIMIT 5

In [ ]:
MATCH (t:Transaction)
RETURN t.Datetime_updated.year as year, count(*) as transactions, sum(t.Price_USD) as total_volume, avg(t.Price_USD) as average_price
ORDER BY year 

In [ ]:
match (t:Trader)
return t.username as username,size((t)-[:BOUGHT]->()) as bought,
         size((t)-[:SOLD]->()) as sold
ORDER BY bought + sold desc limit 5; 

In [ ]:
match (t:Trader)
OPTIONAL MATCH (t)-[:BOUGHT]->(bt)
WITH t, sum(bt.Price_USD) AS boughtVolume
OPTIONAL MATCH (t)-[:SOLD]->(st)
WITH t, boughtVolume, sum(st.Price_USD) AS soldVolume
RETURN t.username as username, t.address as address,boughtVolume, soldVolume
ORDER BY boughtVolume + soldVolume
DESC LIMIT 10

In [ ]:
MATCH (t:Trader)-[:SOLD]->(st:Transaction)-[:FOR_NFT]->(nft)
WHERE st.Price_USD > 100000
MATCH (t)-[:BOUGHT]->(bt:Transaction)-[:FOR_NFT]->(nft)
WHERE st.Datetime_updated_seconds > bt.Datetime_updated_seconds
RETURN t.username as trader, nft.Image_url_1 as nft, st.Datetime_updated_seconds AS soldTime, st.Price_USD AS soldAmount, bt.Datetime_updated_seconds as boughtTime, bt.Price_USD AS boughtAmount,
        st.Price_USD - bt.Price_USD AS difference
ORDER BY difference DESC LIMIT 10

In [4]:
# Network analysis


In [ ]:
CALL gds.graph.create.cypher("nft", 
"MATCH (t:Trader) RETURN id(t) as id",
"MATCH (t1:Trader)-[:BOUGHT]->(t)<-[:SOLD]-(t2:Trader)
 RETURN id(t1) AS source, id(t2) as target, coalesce(sum(t.Price_USD),1) as weight")

In [ ]:
CALL gds.wcc.stats("nft")

In [ ]:
CALL gds.pageRank.stream("nft", {relationshipWeightProperty:'weight'})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).username as username,gds.util.asNode(nodeId).address as address, score
ORDER BY score DESC
LIMIT 10